<a href="https://colab.research.google.com/github/vkjadon/tf/blob/main/tf203-dogsvscats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import os

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [6]:
!ls

drive  sample_data


In [4]:
import glob

In [15]:
data_dir = 'drive/MyDrive/datasets/dog-vs-cat/train/'

In [10]:
!pwd

/content


In [16]:
os.listdir(data_dir)

['dog', 'cat']

In [ ]:
os.listdir(os.path.join(data_dir,'cat'))

In [20]:
for image_class in os.listdir(data_dir):
  print("Class - ", image_class)
  i=0
  for image in os.listdir(os.path.join(data_dir, image_class)):
    i=i+1
  print(i)


Class -  dog
500
Class -  cat
501


In [21]:
import cv2

## Change Path (we can aslo use `cd`)
The below method will store the path in `image_path`. Can aslo directly do by storing the string in the variable as we did for `data_dir`.

In [30]:
image_path=os.path.join(data_dir, 'cat')
print(type(image_path))
print(image_path)

<class 'str'>
drive/MyDrive/datasets/dog-vs-cat/train/cat


##Show list in the files in the above directory

In [29]:
image_list=os.listdir(image_path)
print(image_list)

['cat.10018.jpg', 'cat.10030.jpg', 'cat.10025.jpg', 'cat.1007.jpg', 'cat.1010.jpg', 'cat.10146.jpg', 'cat.10151.jpg', 'cat.10144.jpg', 'cat.10152.jpg', 'cat.10150.jpg', 'cat.1013.jpg', 'cat.10145.jpg', 'cat.1012.jpg', 'cat.10193.jpg', 'cat.10179.jpg', 'cat.10178.jpg', 'cat.10191.jpg', 'cat.10187.jpg', 'cat.10233.jpg', 'cat.10232.jpg', 'cat.10344.jpg', 'cat.10346.jpg', 'cat.10350.jpg', 'cat.10352.jpg', 'cat.10391.jpg', 'cat.1038.jpg', 'cat.10385.jpg', 'cat.10421.jpg', 'cat.10436.jpg', 'cat.10554.jpg', 'cat.10596.jpg', 'cat.10555.jpg', 'cat.10568.jpg', 'cat.10582.jpg', 'cat.10597.jpg', 'cat.10608.jpg', 'cat.10634.jpg', 'cat.10620.jpg', 'cat.10637.jpg', 'cat.10623.jpg', 'cat.10757.jpg', 'cat.10805.jpg', 'cat.10781.jpg', 'cat.10795.jpg', 'cat.10839.jpg', 'cat.10807.jpg', 'cat.10813.jpg', 'cat.10959.jpg', 'cat.10964.jpg', 'cat.10970.jpg', 'cat.11070.jpg', 'cat.11059.jpg', 'cat.11058.jpg', 'cat.11066.jpg', 'cat.11064.jpg', 'cat.11105.jpg', 'cat.11111.jpg', 'cat.11104.jpg', 'cat.11138.jpg', '

In [31]:
import random

In [37]:
random_image=random.choice(os.listdir(image_path))
print(random_image)

cat.9937.jpg


In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

So besides that, when we call flow from directory, this is where we are passing in our actual data and specifying how we want this data to be processed. So we are setting directory equal to train path, which appear we defined the location on disk where our training set was under the train PATH variable. And then we're setting target size equal to 224 by 224.
So if you're working with an image data set that has images of varying sizes, or you just want to scale them up or scale them down, this is how you can specify that to happen. And this will resize all images in your data set to be of this height and width before passing them to our network. Now we are specifying our classes, which are just the classes for the potential labels of our data set. So cat or dog, and we are setting our batch size to 10.
We do the exact same thing for the validation set. And the test set.

Everything is the exact same for both of them, except for where each of these sets live on disk as being specified here under the directory parameter.
And then the only other difference is here for our test batches. We are specifying this shuffle equals false parameter.
Now, this is because whenever we use our test batches later for inference to get our model
to predict on images of cats and dogs after training and validation has been completed,
we're going to want to look at our prediction results in a confusion matrix like we did in a previous video for a separate data set.
And in order to do that, we need to be able to access the unsettled labels for our test
set. So that's why we set shuffle equals false for only this set. For both validation and training sets, we do want the data sets to be shuffled.